# Chapter 1： 波士顿房价预测模型

## 问题描述
通过多元线性回归模型，对波士顿放假进行预测。数据集统计了13种可能影响房价的因素和该类型房屋的均价，期望构建一个基于13个因素进行房价预测的模型。 各指标属性名如下`feature_names = [ 'CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE','DIS','RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV' ]`

## 线性回归模型

假设房价和各影响因素之间能够用线性关系来描述：

$$y = {\sum_{j=1}^Mx_j w_j} + b$$

模型的求解即是通过数据拟合出每个$w_j$和$b$。其中，$w_j$和$b$分别表示该线性模型的权重和偏置。

线性回归模型使用均方误差作为损失函数（Loss），用以衡量预测房价和真实房价的差异，公式如下：

$$MSE = \frac{1}{n} \sum_{i=1}^n(\hat{Y_i} - {Y_i})^{2}$$
注意： 在此处使用均方误差，避免负数的产生，有利于模型的优化。

## 模型构建和训练五步骤：
**数据处理：** 数据导入，数据形状变换，数据集划分，数据归一化，封装成函数  
**模型设计：** 网络结构构建，模型的假设空间，利用模型结构，表达假设关系  
**训练配置：** 设定模型采用的寻解方法，指定计算资源  
**训练过程：** 循环调用训练过程，每轮都包括向前计算，损失函数，向后传播三个步骤。  
**模型保存：** 将训练好的模型进行保存，在模型预测的时候调用


### 数据处理
#### 读入数据

In [1]:
import numpy as np
import json
# 读入训练数据
datafile = 'housing.data'
data = np.fromfile(datafile, sep=' ')
data

#### 数据形状转变
以上输入数据为一维 array， 现在需要将每14项分成一条数据，变成 X * 14 的 array 变量

In [2]:
feature_names = [ 'CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE','DIS', 
                 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV' ]
feature_num = len(feature_names)
length = data.shape
column_num = int(length[0]/feature_num)

data = data.reshape([506, feature_num])
# 注意双斜杠是整除的意思


#### 数据集划分
在本案例中，我们将80%的数据用作训练集，20%用作测试集，实现代码如下。通过打印训练集的形状，可以发现共有404个样本，每个样本含有13个特征和1个预测值。

In [3]:
ratio = 0.8
offset = int(data.shape[0] * ratio)
training_data = data[:offset]
training_data.shape

#### 数据归一化处理

对每个特征进行归一化处理，使得每个特征的取值缩放到0~1之间。这样做有两个好处：一是模型训练更高效；二是特征前的权重大小可以代表该变量对预测结果的贡献度（因为每个特征值本身的范围相同）。

In [4]:
    max_value = training_data.max(axis = 0)
    min_value = training_data.min(axis = 0)
    ave_value = training_data.mean(axis = 0)
    

    for i in range (feature_num):
        data[:,i] = (data[:, i] - ave_value[i])/(max_value[i] - min_value[i])

#### 封装函数

In [5]:
import numpy as np
import json

def Load_Data():
    datafile = 'housing.data'
    data = np.fromfile(datafile, sep=' ')

    feature_names = [ 'CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE','DIS', 
                    'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV' ]
    feature_num = len(feature_names)
    data = data.reshape([data.shape[0] // feature_num, feature_num])

    # 区分训练集和测试集

    ratio = 0.8
    offset = int(data.shape[0] * ratio)
    training_data = data[:offset]


    # 数据归一化
    max_value = training_data.max(axis = 0)
    min_value = training_data.min(axis = 0)
    ave_value = training_data.mean(axis = 0)
    

    for i in range (feature_num):
        data[:,i] = (data[:, i] - ave_value[i])/(max_value[i] - min_value[i])

    


    training_data = data [:offset]
    testing_data =  data [offset:]
    return training_data, testing_data

### 模型设计

模型设计也hi对网络结构的设计，实现模型向前计算。 在此项目中，输入特种 $x$ 有13个分量，输出$y$仅一个分量，则参数权重形状为$13*1$, 因此参数权重，输入特征，和预测结果存在以下关系  
$$z = x*w + b$$  
并通过预测结果和实际结果计算损失函数。  
可定义一个network类来定义以下函数

In [6]:
class Network(object):
    def __init__(self, num_of_weights):
        # 随机产生w的初始值
        # 为了保持程序每次运行结果的一致性，此处设置固定的随机数种子
        np.random.seed(0)
        self.w = np.random.randn(num_of_weights, 1)
        self.b = 0.
        
    def forward(self, x):
        z = np.dot(x, self.w) + self.b
        return z
    
    def loss(self, z, y):
        error = z - y
        num_samples = error.shape[0]
        cost = error * error
        cost = np.sum(cost) / num_samples
        return cost

### 训练过程

通过训练，找到使损失函数值较小的参数（$b$,$w$）。可采用随机梯度下降法。  

**梯度下降法（英语：Gradient descent）是一个一阶最优化算法，通常也称为最陡下降法，但是不该与近似积分的最陡下降法（英语：Method of steepest descent）混淆。 要使用梯度下降法找到一个函数的局部极小值，必须向函数上当前点对应梯度（或者是近似梯度）的反方向的规定步长距离点进行迭代搜索。**  

当函数的梯度下降为零的时候，函数趋向极值  

#### 随机梯度下降法（ Stochastic Gradient Descent）

在上述程序中，每次损失函数和梯度计算都是基于数据集中的全量数据。对于波士顿房价预测任务数据集而言，样本数比较少，只有404个。但在实际问题中，数据集往往非常大，如果每次都使用全量数据进行计算，效率非常低，通俗地说就是“杀鸡焉用牛刀”。由于参数每次只沿着梯度反方向更新一点点，因此方向并不需要那么精确。一个合理的解决方案是每次从总的数据集中随机抽取出小部分数据来代表整体，基于这部分数据计算梯度和损失来更新参数，这种方法被称作随机梯度下降法（Stochastic Gradient Descent，SGD），核心概念如下：

**随机的原因：** 模型会对最后训练的数据有加深记忆，如果样本发生与时间轴有关需要另外考虑。

* min-batch：每次迭代时抽取出来的一批数据被称为一个min-batch。
* batch_size：一个mini-batch所包含的样本数目称为batch_size。
* epoch：当程序迭代的时候，按mini-batch逐渐抽取出样本，当把整个数据集都遍历到了的时候，则完成了一轮训练，也叫一个epoch。启动训练时，可以将训练的轮数num_epochs和batch_size作为参数传入。

In [12]:
import numpy as np

class Network(object):
    def __init__(self, num_of_weights):
        # 随机产生w的初始值
        # 为了保持程序每次运行结果的一致性，此处设置固定的随机数种子
        #np.random.seed(0)
        self.w = np.random.randn(num_of_weights, 1)
        self.b = 0.
        
    def forward(self, x):
        z = np.dot(x, self.w) + self.b
        return z
    
    def loss(self, z, y):
        error = z - y
        num_samples = error.shape[0]
        cost = error * error
        cost = np.sum(cost) / num_samples
        return cost
    
    def gradient(self, x, y):
        z = self.forward(x)
        N = x.shape[0]
        gradient_w = 1. / N * np.sum((z-y) * x, axis=0) 
        gradient_w = gradient_w[:, np.newaxis]
        gradient_b = 1. / N * np.sum(z-y)
        return gradient_w, gradient_b
    
    def update(self, gradient_w, gradient_b, eta = 0.01): #
        self.w = self.w - eta * gradient_w 
        self.b = self.b - eta * gradient_b
            
                
    def train(self, training_data, num_epoches, batch_size=10, eta=0.01):
        n = len(training_data)
        losses = []
        for epoch_id in range(num_epoches):
            # 在每轮迭代开始之前，将训练数据的顺序随机打乱
            # 然后再按每次取batch_size条数据的方式取出
            np.random.shuffle(training_data)
            # 将训练数据进行拆分，每个mini_batch包含batch_size条的数据
            mini_batches = [training_data[k:k+batch_size] for k in range(0, n, batch_size)]
            
            for iter_id, mini_batch in enumerate(mini_batches): # enumerate 函数用法，iter_id 为int 
                #print(self.w.shape)
                #print(self.b)
                x = mini_batch[:, :-1]
                y = mini_batch[:, -1:]
                a = self.forward(x)
                loss = self.loss(a, y)
                gradient_w, gradient_b = self.gradient(x, y)
                self.update(gradient_w, gradient_b, eta)
                losses.append(loss)
                print('Epoch {:3d} / iter {:3d}, loss = {:.4f}'.
                                 format(epoch_id, iter_id, loss))
        
        return losses

# 获取数据
train_data, test_data = Load_Data()

# 创建网络
net = Network(13)
# 启动训练
losses = net.train(train_data, num_epoches=50, batch_size=100, eta=0.1)

# 画出损失函数的变化趋势
plot_x = np.arange(len(losses))
plot_y = np.array(losses)
plt.plot(plot_x, plot_y)
plt.show()

Epoch   0 / iter   0, loss = 0.2948
Epoch   0 / iter   1, loss = 0.2892
Epoch   0 / iter   2, loss = 0.2995
Epoch   0 / iter   3, loss = 0.3779
Epoch   0 / iter   4, loss = 0.6146
Epoch   1 / iter   0, loss = 0.3129
Epoch   1 / iter   1, loss = 0.2351
Epoch   1 / iter   2, loss = 0.2831
Epoch   1 / iter   3, loss = 0.2426
Epoch   1 / iter   4, loss = 0.0872
Epoch   2 / iter   0, loss = 0.2392
Epoch   2 / iter   1, loss = 0.2329
Epoch   2 / iter   2, loss = 0.3048
Epoch   2 / iter   3, loss = 0.1769
Epoch   2 / iter   4, loss = 0.2686
Epoch   3 / iter   0, loss = 0.2558
Epoch   3 / iter   1, loss = 0.2413
Epoch   3 / iter   2, loss = 0.1872
Epoch   3 / iter   3, loss = 0.1861
Epoch   3 / iter   4, loss = 0.2442
Epoch   4 / iter   0, loss = 0.1860
Epoch   4 / iter   1, loss = 0.1754
Epoch   4 / iter   2, loss = 0.2022
Epoch   4 / iter   3, loss = 0.2241
Epoch   4 / iter   4, loss = 0.1511
Epoch   5 / iter   0, loss = 0.1929
Epoch   5 / iter   1, loss = 0.1754
Epoch   5 / iter   2, loss =

Epoch  49 / iter   4, loss = 0.0149


NameError: name 'plt' is not defined

观察上述Loss的变化，随机梯度下降加快了训练过程，但由于每次仅基于少量样本更新参数和计算损失，所以损失下降曲线会出现震荡。

------
**说明：**

由于房价预测的数据量过少，所以难以感受到随机梯度下降带来的性能提升。

------